# Provisioning

## Instances
Instances are sets of vms of various types that can be provisioned across the GreenLake for private cloud stack and a wide range of services like MySQL, Redis, ElasticSearch, PostgreSQL, Tomcat, nginx, Confluence, Jenkins, and more. There are a few important concept differentiators between what we call an instance and what amazon calls an instance. In GreenLake an instance can represent many vms that are of a set. For example. If you wanted to spin up a Mongo sharded replicaset, that requires 7 virtual machines. In GreenLake for private cloud we represent this as a singular instance with a specified layout and then represent all the associated services running within that virtual machine. The serverId represents the compute resource it was provisioned onto, (i.e. the virtual machine).

Get all instances
- GET /api/instances

Get a specific instance
- GET /api/instances/:id

Make sure to report your token here again in order to set the variable correctly and to execute the cell once updated.

In [ ]:
username="student{{ STDID }}" 


##Initializing Variables here
ACCESS_TOKEN=$(cat token.txt)
echo "your api token is: $ACCESS_TOKEN"


API_URL="https://client.greenlake.hpe.com/api/iac-vmaas/v1"

space="HPEDEV HackShack workshops"
location="HPE"
stdid={{ STDID }}

Let's start by checking how many instances are running here

In [ ]:
curl -i -X GET \
  '$API_URL/instances?space="$space"&location="$location"' \
  -H "Authorization: $ACCESS_TOKEN" |jq -r

We will need the student group id as well, let 's get it as a variable now.


In [ ]:
echo "Getting your group Id now"
groupid=$(curl -s "$API_URL/groups?space="$space"&location="$location"&name=student{{ STDID }}" \
-H "Authorization: $ACCESS_TOKEN" |jq -r '.groups[].id')
echo "your student groupid is: $groupid"

## Create now your first instance!

You will deploy here a centos 7 VM named after your studentname ("name": "api-vm-student{{ STDID }})

### Creating an instance
- POST /api/instances

Payload: image  here


In [ ]:
curl -i -X GET \
  '$API_UR/instances?space="$space"&location="$location"&max=25&offset=0&name=string&phrase=string&createdBy=string&showDeleted=false&instanceType=0&status=string&deleted=true&labels=string&tags=string' \
  -H "Authorization: $ACCESS_TOKEN"
  
  
  
curl -i -X POST \
  '$API_UR/instances?space="$space"&location="$location"' \
  -H "Authorization: $ACCESS_TOKEN" \
  -H 'Content-Type: application/json' \
  -d '{
    "zoneId": 0,
    "instance": {
      "name": "student{{ STDID}}-vm",
      "site": {
        "id": $groupid
      },
      "instanceType": {
        "code": "vmware"
      },
      "layout": {
        "id": 0
      },
      "plan": {
        "id": 0
      },
      "instanceContext": "string",
      "powerScheduleType": 0,
      "shutdownDays": 0,
      "expireDays": 0,
      "environmentPrefix": "string"
    },
    "volumes": [
      {
        "id": -1,
        "rootVolume": true,
        "name": "root",
        "size": 0,
        "storageType": 0,
        "datastoreId": "auto"
      }
    ],
    "layoutSize": 1,
    "networkInterfaces": [
      {
        "network": {
          "id": 0
        },
        "networkInterfaceTypeId": 0
      }
    ],
    "config": {
      "template": 0,
      "resourcePoolId": "string",
      "noAgent": false,
      "smbiosAssetTag": null,
      "vmwareFolderId": "string",
      "hostId": null,
      "vmwareDomainName": null,
      "vmwareCustomSpec": null,
      "nestedVirtualization": null,
      "createUser": false,
      "customOptions": {
        "<OptionTypeFieldNameForText/Password/Secret>": "string",
        "<OptionTypeFieldNameForCheckBox>": true,
        "<OptionTypeFieldNameForTypeAhead/SelectList>": {
          "name": "string",
          "value": "string"
        }
      }
    },
    "evars": [
      {
        "name": "string",
        "value": "string"
      }
    ],
    "tags": [
      {
        "name": "string",
        "value": "string"
      }
    ],
    "labels": [
      "string"
    ],
    "workflow": {
      "taskSetId": 0
    }
  }'

In [ ]:
curl -s -X --request POST "$API_URL/api/instances" \
-H "Authorization: BEARER $ACCESS_TOKEN" \
-H "Content-Type: application/json" \
--data-raw "{ "zoneId": 1, "instance": { "name": "vm-student{{ STDID }}-api", "site": { "id": $groupid }, "instanceType": { "code": "glhc-vanilla-centos" }, "layout": { "id": 1159 }, "plan": { "id": 402 } }, "volumes": [ { "name": "root", "size": 15, "datastoreId": "auto" } ], "networkInterfaces": [ { "network": { "id": 7 }, "networkInterfaceTypeId": 4 } ], "config": { "resourcePoolId": 2 }, "tags": [ {"name": "api-provisioned", "value": "True"} ]  }"

Once created and as for any Virtual machine, many operations are available to you:
- Update
- Stop
- Start
- Restart
- Delete
- Backup
- Clone
- Resize

Let's see these in details now.

Let's now get the the VM id to be used later for managing it. We'll make it a variable for ease of use.

In [ ]:
student{{ STDID }}vmid=$(curl -s --location --request GET "$API_URL/api/instances" \
-H "Authorization: BEARER $ACCESS_TOKEN" |jq -r '.instances[0].id')


echo $student{{ STDID }}vmid

### Updating an instance

- PUT /api/instances/:id

**Note: Site is alias of Group**

- --name VALUE Name
- --description VALUE Description
- --environment VALUE Environment
- --group GROUP Group Name or ID
- --labels [LIST] Labels (keywords) in the format 'foo, bar'
- --tags LIST Tags in the format 'name:value, name:value'. This will add and remove tags.
- --add-tags TAGS Add Tags in the format 'name:value, name:value'. This will only add/update tags.
- --remove-tags TAGS Remove Tags in the format 'name, name:value'. This removes tags, the :value component is optional and must match if passed.
- --power-schedule-type ID Power Schedule Type ID
- --owner USER Owner Username or ID



Here, we will simply update the description field:
It should be set to null as we did not specify it at the instance creation time

In [ ]:
curl -s --location --request GET "$API_URL/api/instances/" \
-H "Authorization: BEARER $ACCESS_TOKEN" | jq -r '.instances[0].description[0]'

Let's Modify it now

In [ ]:
curl -L -X PUT "$API_URL/api/instances/$student{{ STDID }}vmid" \
-H "Authorization: Bearer $ACCESS_TOKEN " \
-H 'Content-Type: application/json' \
--data-raw '{ "instance": {
  "description": "Description updated by API call by student{{ STDID}}"
  }}'

### Let's check the difference now : Description field should no longer be null

In [ ]:
curl -s --location --request GET "$API_URL/api/instances/" \
-H "Authorization: BEARER $ACCESS_TOKEN" | jq -r |grep description

### Stop an instance

- PUT /api/instances/:id/stop?server=true&muteMonitoring=false

- --mute-monitoring [on|off] Mute monitoring. Default is off.
- -y, --yes Auto Confirm
- -q, --quiet No Output, do not print to stdout

In [ ]:
curl -s --request PUT "$API_URL/api/instances/$student{{ STDID }}vmid/stop?server=true&muteMonitoring=false" \
-H "Authorization: BEARER $ACCESS_TOKEN" 

### Start an instance

- PUT /api/instances/:id/start?server=true

- -y, --yes Auto Confirm
- -q, --quiet No Output, do not print to stdout

In [ ]:
curl -s --request PUT "$API_URL/api/instances/$student{{ STDID }}vmid/start?server=true&muteMonitoring=true" \
-H "Authorization: BEARER $ACCESS_TOKEN"

### Restart an instance

- PUT /api/instances/:id/restart?server=true&muteMonitoring=true

- --mute-monitoring [on|off] Mute monitoring. Default is on.
- -y, --yes Auto Confirm
- -q, --quiet No Output, do not print to stdout

In [ ]:
curl -s --request PUT "$API_URL/api/instances/$student{{ STDID }}vmid/restart?server=true&muteMonitoring=true" \
-H "Authorization: BEARER $ACCESS_TOKEN"

### Snapshot an instance   
- PUT /api/instances/:id/snapshot
-d '{snapshot}
--name VALUE Name
--description VALUE Description

## Caution !!  The snapshot takes a few minutes to complete. Please be patient...

In [ ]:
curl -s --request PUT "$API_URL/api/instances/$student{{ STDID }}vmid/snapshot" \
-H "Authorization: BEARER $ACCESS_TOKEN" \
-H "Content-Type: application/json" \
-d '{ "snapshot": { "name": "api-vm-student{{ STDID }}-snap", "description": "A snapshot created via VMAAS API Call for student{{ STDID }} instance" }}'

### List now your Instance Snapshot
- GET/api/instances/:id/snapshots


In [ ]:
curl -s --request GET "$API_URL/api/instances/$student{{ STDID }}vmid/snapshots" \
-H "Authorization: BEARER $ACCESS_TOKEN" \
-H "Content-Type: application/json" \

### You can also clone or resize  an instance

- PUT /api/instances/:id/clone

- PUT /api/instances/:id/resize

Payload:

{
  "instance": {
    "id": 482,
    "plan": {
      "id": 419
    }
  },
  "volumes": [
    {
      "id": 643,
      "rootVolume": true,
      "name": "Hard disk 1",
      "size": 10,
      "sizeId": null,
      "storageType": 1,
      "datastoreId": "auto"
    }
  ],
  "deleteOriginalVolumes": true
}

### What about accessing the vm now ? 
Let's get the details for our centos instance. An IP address might help to connect. Some crededntials too.

In [ ]:
student{{ STDID }}vmip=$(curl -s --location --request GET "$API_URL/api/instances/ \
-H "Authorization: BEARER $ACCESS_TOKEN" |jq -r '.instances[0].connectionInfo[0].ip')

echo $student{{ STDID }}vmip

#### Check if vm is reachable 

In [ ]:
ping $student{{ STDID }}vmip

####  Connect now to the vm

In [ ]:
ssh student1701@$student{{ STDID }}vmip

## Let's do some Cleanup now

use DELETE $serverUrl/api/instances/:id

In [ ]:
curl -s --location --request DELETE "$API_URL/api/instances/74" \
-H "Authorization: BEARER $ACCESS_TOKEN"

## Summary:
In this lab, you created your first instance leveraging the HPE GreenLake VMAAS API. You also managed to stop, start, restart this instance. You also created your first snapshot before accessing the vm.

<br><br>

## <i class="fas fa-2x fa-map-marker-alt" style="color:#008000;"></i>&nbsp;&nbsp;Next Steps

Lab 3: Tasks

</br>
 <a href="1-WKSHP-VMAAS-Authentification.ipynb" target="New" title="Back: Authentification"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#008000;color:#fff;position:relative;width:10%; height: 30px;float: left;"><b>Back</b></button></a>
 <a href="3-WKSHP-WKSHP-VMAAS-Tasks.ipynb#sc" target="New" title="Next:Tasks"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#008000;color:#fff;position:relative;width:10%; height: 30px;float: right;"><b>Next</b></button></a>


<h2>Next&nbsp;&nbsp;&nbsp;&nbsp;<a href="3-WKSHP-WKSHP-VMAAS-Tasks.ipynb#sc" target="New" title="Next: Tasks"><i class="fas fa-chevron-circle-right" style="color:#008000;"></i></a></h2>